In [1]:
import pandas as pd
import geopandas as gpd
import osmnx as ox
import folium
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from shapely.geometry import Point, Polygon, MultiPolygon
from shapely.ops import nearest_points


In [2]:
# Workaround to fix chrome issue where folium won't plot maps with a large number of layers
# See comment by dstein64 at: https://github.com/python-visualization/folium/issues/812

import base64
def _repr_html_(self, **kwargs):
    html = base64.b64encode(self.render(**kwargs).encode('utf8')).decode('utf8')
    onload = (
        'this.contentDocument.open();'
        'this.contentDocument.write(atob(this.getAttribute(\'data-html\')));'
        'this.contentDocument.close();'
    )
    if self.height is None:
        iframe = (
            '<div style="width:{width};">'
            '<div style="position:relative;width:100%;height:0;padding-bottom:{ratio};">'
            '<iframe src="about:blank" style="position:absolute;width:100%;height:100%;left:0;top:0;'
            'border:none !important;" '
            'data-html={html} onload="{onload}" '
            'allowfullscreen webkitallowfullscreen mozallowfullscreen>'
            '</iframe>'
            '</div></div>').format
        iframe = iframe(html=html, onload=onload, width=self.width, ratio=self.ratio)
    else:
        iframe = ('<iframe src="about:blank" width="{width}" height="{height}"'
                  'style="border:none !important;" '
                  'data-html={html} onload="{onload}" '
                  '"allowfullscreen" "webkitallowfullscreen" "mozallowfullscreen">'
                  '</iframe>').format
        iframe = iframe(html=html, onload=onload, width=self.width, height=self.height)
    return iframe

folium.branca.element.Figure._repr_html_ = _repr_html_


In [3]:
def gridify_polygon(poly,grid_spacing):
    # creates a cartesian grid inside polygon with the input grid_spacing
    # poly: polygon which we want a grid inside
    # grid_spacing: spaceing in lattitude/longitude degrees
    poly_xmin,poly_ymin,poly_xmax,poly_ymax = poly.geometry.total_bounds

    cols = list(np.arange(poly_xmin,poly_xmax+grid_spacing,grid_spacing))
    rows = list(np.arange(poly_ymin,poly_ymax+grid_spacing,grid_spacing))
    rows.reverse()

    polygons = []
    for x in cols:
        for y in rows:
            polygons.append( Polygon([(x,y), (x+grid_spacing, y), (x+grid_spacing, y-grid_spacing), (x, y-grid_spacing)]) )

    grid = gpd.GeoDataFrame({'geometry':polygons})

    grid['isin_poly'] = grid.apply(lambda row: row['geometry'].centroid.within(poly.geometry[0]), axis=1)
    poly_grid = grid[grid.isin_poly == True]
    poly_grid.crs = {'init': 'epsg:4326', 'no_defs': True}
    poly_grid = poly_grid.drop(['isin_poly'], axis = 1)
    
    # Calculate the polygon areas in km
    poly_grid_cart = poly_grid.copy()
    poly_grid_cart = poly_grid_cart.to_crs({'init': 'epsg:3857'})
    poly_grid_cart['poly_area_km'] = poly_grid_cart['geometry'].area/ 10**6
    # Store polygon area
    poly_grid['poly_area_km'] = poly_grid_cart['poly_area_km']
    
    # 
    poly_grid = poly_grid.reset_index()
    return poly_grid

def amenity_in_polygon(amenity_points,poly):
    # returns the amenities that are inside the given polygon
    # When there are zero amenities within the interrogation region, the function returns an empty dataframe as
    # as expected, but also prints out a lot of errors. not a huge issue but annoying.
    # Maybe implement a test for if empty, return 0
    # Example use:
    #         amenity_in_polygon(food_amenities,city_grid.geometry.iloc[38])
    
    # Generate boolean list of whether amenity is in polygon
    indices = amenity_points.apply(lambda row: row['geometry'].within(poly), axis=1)
    if not any(indices): # If all indices are false
        return pd.DataFrame(columns=['A']) # return empty dataframe (not sure what is best to output here )
    else:
        return amenity_points[amenity_points.apply(lambda row: row['geometry'].within(poly), axis=1)]

def avg_dist_to_amenities(interrogation_point,amenity_df,n):
    # calculates the mean distance of the n nearest amenities to the interrogation point
    # If there are less than n amenities in the search it'll just return the average of the known amenities.
    # Example: avg_dist_to_amenities(city_grid.geometry.iloc[39],food_amenities,5)
    dist_to_amenity = amenity_df['geometry'].apply(lambda x: x.distance(interrogation_point))
    dist_to_amenity.sort_values(inplace=True)
    dist_to_amenity[:5]
    if len(dist_to_amenity) >= n:
        return dist_to_amenity[:n].mean()
    elif len(dist_to_amenity) == 0:
        return np.nan
    else:
        return dist_to_amenity.mean()
    
    
# Generate features dataframe by finding the count of each unique amenity in each region
def features_density(interrogation_grid,osm_features,targets):
    # Calculate feature and target density inside a series of polygons
    # INPUTS
    # ------
    # Interrogation grid: list of polygons in which to calculate density of features and targets
    # osm_features: gdf of amenities in area retrieved from OSM
    # targest: gdf of target locations
    # OUTPUTS
    # -------
    # cleaned_df: contains the density of features and targets in the interrogation grid
    # create new cleaned df that will store features and target data
    cleaned_df = interrogation_grid.copy()
    cleaned_df = cleaned_df.reset_index()
    cleaned_df['bike_rental_density'] = 0
    cleaned_df = cleaned_df.reindex(cleaned_df.columns.tolist() + amenity_names, axis=1) 


    # loop through grid points and populate features.
    for index, row in cleaned_df.iterrows():
        grid_pt = cleaned_df.geometry.iloc[index]
        amenities_in_grid = amenity_in_polygon(osm_features,grid_pt)

        # fill amenity rows with counts inside each polygon
        if len(amenities_in_grid) > 0:
            amenity_counts = amenities_in_grid['amenity'].value_counts()
            for val, cnt in amenity_counts.iteritems():
                # test if value is in list of features that are selected for ML model
                if val in amenity_names:
                    cleaned_df[val].iloc[index] = cnt / cleaned_df.poly_area_km.iloc[index]

        # add target column for bike rentals
        bike_rentals_in_grid = amenity_in_polygon(targets,grid_pt)
        if len(bike_rentals_in_grid) > 0:
            cleaned_df['bike_rental_density'].iloc[index] = len(bike_rentals_in_grid) / cleaned_df.poly_area_km.iloc[index]
        else:
            cleaned_df['bike_rental_density'].iloc[index] = 0

    # remove nan values
    cleaned_df[amenity_names] = cleaned_df[amenity_names].fillna(0)
    # remove unecessary columns
    cleaned_df = cleaned_df.drop(columns = ['level_0','index'])
    # relable as density 
    new_names = [name + '_density' for name in amenity_names]
    cleaned_df.rename(columns = dict(zip(amenity_names, new_names)), inplace=True)
    
    return cleaned_df

In [4]:
place = 'Glasgow City, Scotland, UK'

filepath = '/Volumes/ZHANG MBA/Strava data/Datasets_1/Strava Metro Assorted/safeguarded-release-strava-metro-glasgow-2015-all-extract/'

# glasgow_nodes_2015 = gpd.read_file(filepath + 'glasgow_1k_2015_nodes.shp')
glasgow_edges_2015 = gpd.read_file(filepath + 'glasgow_1k_edges_2015.shp')
# glasgow_edges_polygon = gpd.read_file(filepath + 'glasgow_1k_od_2015_polygons.shp')

# node_alignment = pd.read_csv(filepath + 'glasgow_2015_ride_node_alignment_hourly.csv')
# total_node_count = pd.read_csv(filepath + 'glasgow_2015_ride_nodes_rollup_total.csv')
# month_node_count = pd.read_csv(filepath + 'glasgow_2015_ride_node_rollup_month_total.csv')
# hourly_node_count = pd.read_csv(filepath + 'glasgow_2015_ride_node_alignment_hourly.csv')

# hourly_edge_count = pd.read_csv(filepath + 'glasgow_2015_ride_edge_alignment_hourly.csv')
# month_edge_count = pd.read_csv(filepath + 'glasgow_2015_ride_edge_rollup_month_total.csv')
total_edge_count = pd.read_csv(filepath + 'glasgow_2015_ride_edge_rollup_total.csv')


In [5]:
city = ox.gdf_from_place(place)

# generate city grid
city_grid = gridify_polygon(city,0.01)

# Get intersection and streets from graph
graph = ox.graph_from_place(place)
nodes, streets = ox.graph_to_gdfs(graph)
# Some columns contain lists. Convert these to tupes so that they are hashable and can be merged with other dfs
streets['highway'] = streets['highway'].apply(lambda x: tuple(x) if type(x) == list else x)
streets['osmid'] = streets['osmid'].apply(lambda x: tuple(x) if type(x) == list else x)


city

,geometry,place_name,bbox_north,bbox_south,bbox_east,bbox_west
0,"POLYGON ((-4.39320 55.88915, -4.39074 55.88859...","Glasgow City, Scotland, United Kingdom",55.929639,55.781277,-4.071717,-4.393201


In [60]:
# merge dataframes into a single one for testing
df1 = glasgow_edges_2015[['id','osm_id','geometry']].merge(total_edge_count[['edge_id','tathcnt','tactcnt','tcmtcnt']], left_on='id', right_on='edge_id', how='inner')
df2 = df1.merge(streets[['u','v','osmid','highway','oneway','length']],left_on='osm_id', right_on='osmid', how='inner')

df2 = df2.drop_duplicates(subset='edge_id', keep='first')
df2 = df2.drop(columns=['edge_id','osmid'])

strava_street_counts = df2.copy()
strava_street_counts.head()



,id,osm_id,geometry,tathcnt,tactcnt,tcmtcnt,u,v,highway,oneway,length
0,54,1762,"LINESTRING (-4.31069 55.84956, -4.31029 55.84945)",140,290,215,1637120075,2415333077,motorway_link,True,28.033
2,55,1762,"LINESTRING (-4.31029 55.84945, -4.30996 55.849...",15,20,20,1637120075,2415333077,motorway_link,True,28.033
4,65,1801,"LINESTRING (-4.25000 55.87015, -4.24756 55.869...",25,30,20,1715259260,27456222,motorway,True,32.027
5,66,1801,"LINESTRING (-4.24447 55.86965, -4.24265 55.86944)",35,40,30,1715259260,27456222,motorway,True,32.027
6,105,1876,"LINESTRING (-4.31275 55.85278, -4.31288 55.852...",470,3870,3190,2285131525,3898424564,primary,True,244.301


28500

In [102]:
# sum athlete counts in each polygon and get total street length
def street_features(city_grid,strava_street_counts):
    highway_types = ['residential', 'footway', 'service', 'tertiary', 'unclassified',
           'primary', 'cycleway', 'secondary', 'pedestrian', 'path',
           'primary_link', 'steps', 'motorway_link', 'track', 'motorway',
           'secondary_link', 'living_street', 'tertiary_link', 'trunk']
    length_names = [name + '_total_length' for name in highway_types]

    # Initialize out_df that stores strava target and feature data
    out_df = city_grid.copy()
    out_df['grid_tactcnt'] = 0
    out_df = out_df.reindex(out_df.columns.tolist() + length_names, axis=1) 


    for index, row in out_df.iterrows():
        grid_pt = out_df.geometry.iloc[index]
        # Generate boolean list of edge  is in polygon
        indices = strava_street_counts.apply(lambda row: grid_pt.contains(row['geometry']), axis=1)

        # Calculate stats of interrogation polygon
        poly_cyclist_stats = strava_street_counts[indices].groupby('highway').sum()
        # store target data
        out_df['grid_tactcnt'].iloc[index] = poly_cyclist_stats.tactcnt.sum()
        # store feature data
        for c,h_type in enumerate(highway_types):
            try:
                out_df[length_names[c]].iloc[index] = poly_cyclist_stats.length.loc[h_type]
            except:
                out_df[length_names[c]].iloc[index] = np.nan
    return out_df

glasgow_cyclist_count = street_features(city_grid,strava_street_counts)


In [103]:
glasgow_cyclist_count.head()

,index,geometry,poly_area_km,grid_tactcnt,residential_total_length,footway_total_length,service_total_length,tertiary_total_length,unclassified_total_length,primary_total_length,...,path_total_length,primary_link_total_length,steps_total_length,motorway_link_total_length,track_total_length,motorway_total_length,secondary_link_total_length,living_street_total_length,tertiary_link_total_length,trunk_total_length
0,17,"POLYGON ((-4.38320 55.92128, -4.37320 55.92128...",2.211268,3145,2119.092,298.594,NaN,758.387,597.108,NaN,...,NaN,NaN,NaN,NaN,350.943,NaN,NaN,NaN,NaN,NaN
1,18,"POLYGON ((-4.38320 55.91128, -4.37320 55.91128...",2.210698,12030,1254.775,658.253,1343.275,NaN,753.804,391.569,...,NaN,295.426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,19,"POLYGON ((-4.38320 55.90128, -4.37320 55.90128...",2.210128,30225,3960.720,199.924,NaN,1765.377,NaN,NaN,...,NaN,282.124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20,"POLYGON ((-4.38320 55.89128, -4.37320 55.89128...",2.209559,80595,2123.290,894.764,587.444,176.830,274.247,735.878,...,NaN,NaN,77.569,NaN,NaN,NaN,NaN,30.76,NaN,NaN
4,23,"POLYGON ((-4.38320 55.86128, -4.37320 55.86128...",2.207852,3755,2147.768,148.325,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
m = folium.Map([city.geometry.centroid.y, city.geometry.centroid.x],
               zoom_start=11,
               tiles="CartoDb positron")

style_city = {'color':'#ebc923 ', 'fillColor': '#ebc923 ', 'weight':'1', 'fillOpacity' : 0.1}
folium.GeoJson(city, style_function=lambda x: style_city).add_to(m)

style_region = {'color':'#1FFD09 ', 'fillColor': '#1FFD09 ', 'weight':'1', 'fillOpacity' : 0.1}
folium.GeoJson(city_grid.geometry.iloc[64], style_function=lambda x: style_region).add_to(m)

# plot Roads
style_edges = {'color':'#1198f7', 'weight':'1'}
folium.GeoJson(df[indices], style_function=lambda x: style_edges).add_to(m)

style_region = {'color':'#1FFD09 ', 'fillColor': '#1FFD09 ', 'weight':'1', 'fillOpacity' : 0.1}
    
m

In [ ]:
streets[streets.osmid == 1762]